In [1]:
!pip3 install torch torchvision torchaudio

  Using cached torchaudio-2.7.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached torchaudio-2.7.1-cp312-cp312-macosx_11_0_arm64.whl (1.8 MB)


In [2]:
!pip install opencv-python

  Using cached numpy-2.2.6-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.247.1 requires numpy==1.26.4, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [3]:
!pip install albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 39.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [albumentations]m [opencv-python-headless]


In [4]:
!pip install lungs-segmentation==1.1.1 --no-deps

In [5]:
# 이미지 경로
img_path = './data/train/0a19fb3c-36ef-401b-9deb-77f4f86d5d80.jpg'

In [6]:
#resnet34

import cv2
import numpy as np
import torch
from lungs_segmentation.inference import inference
from lungs_segmentation.pre_trained_models import create_model

# 모델 생성 및 CPU에 올림
model = create_model('resnet34').to(device='mps')
model = model.to('mps')

# 추론 실행
image, mask = inference(model, img_path)

# 결과 시각화
cv2.imwrite('masked_output_resnet34.png', mask[0] * 255)

/Users/skku_aws23/Desktop/cursor_project/github/rsna-pneumonia-detection-challenge/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /Users/skku_aws23/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


100%|██████████| 83.3M/83.3M [00:01<00:00, 59.6MB/s]


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /Users/skku_aws23/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 60.2MB/s]


Downloading: "https://github.com/alimbekovKZ/lungs_segmentation/releases/download/1.0.0/resnet34.pth" to /Users/skku_aws23/.cache/torch/hub/checkpoints/resnet34.pth


100%|██████████| 98.6M/98.6M [00:03<00:00, 28.6MB/s]
[ WARN:0@9.846] global loadsave.cpp:275 findDecoder imread_('./data/train/0a19fb3c-36ef-401b-9deb-77f4f86d5d80.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'min'

In [28]:
#densenet121

import cv2
import numpy as np
import torch
from lungs_segmentation.inference import inference
from lungs_segmentation.pre_trained_models import create_model

# 모델 생성 및 CPU에 올림
model = create_model('densenet121').to(device='mps')
model = model.to('mps')

# 추론 실행
image, mask = inference(model, img_path)

# 결과 시각화
cv2.imwrite('masked_output_densenet121.png', mask[0] * 255)

True

In [26]:
mask

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], shape=(2, 512, 512), dtype=uint8)

In [ ]:
# 마스크 중 첫 번째 채널 사용 (좌/우 폐 마스크 합치기)
lung_mask = mask[0] + mask[1]

# resize to image 크기
lung_mask_resized = cv2.resize(lung_mask.astype("uint8"), (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

# 마스크를 3채널로 확장
lung_mask_3ch = np.stack([lung_mask_resized]*3, axis=-1)  # shape: (H, W) → (H, W, 3)

# 곱셈: 마스크 적용
masked_image = image * lung_mask_3ch  # (H, W, 3) * (H, W, 3)

# 결과 저장
cv2.imwrite("lung_only.png", masked_image)

In [21]:
# 마스크 중 첫 번째 채널 사용 (왼쪽/오른쪽 폐 중 하나 또는 둘 합치기)
lung_mask = mask[0] + mask[1]  # 또는 np.max(mask, axis=0) 도 가능

# 원본 이미지 크기로 resize
lung_mask_resized = cv2.resize(lung_mask.astype("uint8"), (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

# 마스크 적용
masked_image = image * lung_mask_resized

# 결과 저장
cv2.imwrite("lung_only.png", masked_image)

ValueError: operands could not be broadcast together with shapes (512,512,3) (512,512) 